In [ ]:
# %pip install -q langgraph langchain langchain-openai langchain-community tavily-python pygraphviz pytz

# Agentic System for Travel Planning

## System Components and Flow

The system consists of seven interconnected agents that collaborate for comprehensive travel planning.

### System Architecture
<img src="architecture.png" alt="architecture" width="450"/>

### Agent Descriptions
1. **User Guide Agent**
   - Gathers essential travel details (e.g. dates, destinations, preferences)
   - Determines planning path (new itinerary vs refinement)
   - Handles user queries through conversational interaction

2. **Destination Planner Agent**
    - Creates initial travel plan with recommended locations
    - Provides foundation for transport and accommodation planning

3. **Transport Advisor Agent**
    - Generates search queries for best travel tickets
    - Focuses on price-performance balance for tickets
    - Provides ticket availability information for different dates

4. **Accommodation Advisor Agent**
    - Generates search queries for suitable lodging options
    - Evaluates price-performance balance for accommodations
    - Ensures alignment with overall travel timeline and destinations

5. **Itinerary Planner Agent**
    - Combines transport and accommodation information with initial plan
    - Creates complete, optimized itinerary

6. **Itinerary Researcher Agent**
    - Generates queries for itinerary refinements
    - Researches specific user requests

7. **Itinerary Optimizer Agent**
    - Modifies existing itinerary
    - Leverages research findings to optimize itinerary


## Imports
Essential imports for system functionality:
- **langgraph**: Graph creation and state management 
- **langchain**: LLM integration and messaging
- **tavily**: Web search capabilities
- **utility imports**: datetime, typing, re, etc.
- **pydantic**: Data validation and settings management

In [ ]:
# Standard library imports
import re
import operator
from typing import TypedDict, Annotated, List
from datetime import datetime

# Third-party imports
import pytz
from dotenv import load_dotenv
from pydantic import BaseModel
from IPython.display import Image, display

# LangChain imports
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

# API clients
from tavily import TavilyClient

## Initializing Core Components

### Core Setup
- **TavilyClient**: Web search client for information gathering
- **ChatOpenAI**: GPT-4 language model (supporting structured responses for search queries) with temperature=0 for consistent outputs.
- **MemorySaver**: State management for persisting conversation context

### Data Models
- **Queries**: BaseModel for search request formatting
- **AgentState**: TypedDict for managing agent system state

In [ ]:
# Core Setup
load_dotenv()
tavily = TavilyClient()
model = ChatOpenAI(model="gpt-4o", temperature=0)
memory = MemorySaver()

# Data models
class Queries(BaseModel):
    """Model for structured search requests"""
    queries: List[str]

class AgentState(TypedDict):
    """State management for the agentic system"""
    messages: Annotated[list[AnyMessage], operator.add]
    task: str
    basic_plan: str
    search: Annotated[list, operator.add]
    research: str

## Agent Prompts
Defined system prompts for each agent

In [ ]:
user_guide_prompt = f"""
You are a travel assistant helping travelers plan or/and revise their itineraries.
You have two capabilities: planning and refining itineraries.

1. Planning: Your task is to chat with travelers to gather essential details, including:
- Departure location
- Destination(s)
- Possible travel date(s) (Note: The current date and time is {datetime.now(pytz.utc).strftime('%Y-%m-%d %H:%M:%S %Z%z')})
- Duration of the trip
- Any specific requests or preferences

Once you have all the required information, return the following structured response:

Plan:
Departure Location: [Starting location]
Destinations: [Destination(s)]
Time: [Possible travel dates including year]
Duration: [Total duration of the trip]
Traveler's requests: [Specific requests or preferences]

2. Refinement: If a traveler requests a refinement to an itinerary that was previously provided by you or them, 
first check if you can handle it with your knowledge. If you decide that you can handle it, directly provide the refined itinerary. 
Otherwise (if a web search is necessary), return the following structured response:

Refine:
Traveler's Request: [What to edit]
Itinerary: [The itinerary in question]
"""

destination_planner_prompt = """
You are a destination recommender for travelers. You will plan a simple itinerary that includes destinations based on the given plan.
For each general location, you should recommend three specific destinations.

Return the following structured response combining the plan with the recommended destinations: 

Plan:
Departure Location: [Starting location]
Destinations: {[Country1]: {[City1]: [Destination1, Destination2, Destination3]}}
Time: [Possible travel dates including year]
Duration: [Total duration of the trip]
Traveler's requests: [Specific requests or preferences]
"""

transport_advisor_prompt = """
You are a transport assistant responsible for generating search queries to find the best price-performance travel tickets.
The travel plan will be optimized according to the ticket information you provide.
Take the given plan into account when crafting the queries.
Only generate queries—do not provide explanations.
"""

accommodation_advisor_prompt = """
You are an accommodation assistant tasked with gathering essential information to find price-performance accommodations for the given travel plan.
Generate search queries to find relevant accommodation details that match the plan.
The travel plan will be optimized according to the accommodation information you provide.
Only generate queries—do not provide explanations.
"""

itinerary_planner_prompt = """
You are a travel assistant responsible for creating a complete, optimized itinerary.
Use the given basic travel plan, ticket, and accommodation details to finalize the itinerary.
Select accommodations and tickets that offer the best balance between price and quality for the final plan.

Only return the detailed itinerary—do not provide explanations.
"""

itinerary_researcher_prompt = """
You are a research assistant responsible for generating search queries to make the necessary changes related to the given traveler's request(s).
Take into account the given itinerary and traveler's request(s) when crafting the queries.

Only generate queries—do not provide explanations.
"""

itinerary_optimizer_prompt = """
You are an itinerary optimizer responsible for editing the given itinerary according to the traveler's request using the provided research results.

Only return the edited itinerary—do not provide explanations.
"""


## Agent Node Functions
Implementation of agent behaviors and logic

In [ ]:
def user_guide_node(state: AgentState):

    messages = [SystemMessage(content=user_guide_prompt)] + state['messages']

    message = model.invoke(messages)

    if any(pattern in message.content for pattern in ["Plan:", "Refine:"]):
        return {"task": message.content}

    return {"messages": [message]}

def exists_action(state: AgentState):

    task = state.get("task", "")

    if re.search(r"Plan:", task):
        return "plan"
    elif re.search(r"Refine:", task):
        return "refine"
    else:
        return "assistant"

def destination_planner_node(state: AgentState):
    messages = [SystemMessage(content=destination_planner_prompt),
                HumanMessage(content=state['task'])]

    response = model.invoke(messages)

    return {"basic_plan": response.content}

def transport_advisor_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=transport_advisor_prompt),
        HumanMessage(content=f"{state['basic_plan']}")
    ])

    transport_search = []
    for q in queries.queries:
        response = tavily.search(query=q,
                                 max_results=2,
                                 include_answer="basic",
                                 )

        transport_search.append(response["answer"])
    return {"search": [transport_search]}

def accommodation_advisor_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=accommodation_advisor_prompt),
        HumanMessage(content=f"{state['basic_plan']}")
    ])

    accommodation_search = []
    for q in queries.queries:
        response = tavily.search(query=q,
                                 max_results=2,
                                 include_answer="basic",
                                 )

        accommodation_search.append(response["answer"])
    return {"search": [accommodation_search]}


def itinerary_planner_node(state: AgentState):
    messages = [
        SystemMessage(content=itinerary_planner_prompt),
        HumanMessage(content=f"""{state['basic_plan']}
                     \n\nHere is the accommodation and ticket info:\n\n{state['search']}""")]
    
    response = model.invoke(messages)
    return {"messages": [response]}

def itinerary_researcher_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=itinerary_researcher_prompt),
        HumanMessage(content=state['task'])
    ])

    research = []
    for q in queries.queries:
        response = tavily.search(query=q,
                                 max_results=2,
                                 include_answer="basic",
                                 )

        research.append(response["answer"])
    return {"research": research}

def itinerary_optimizer_node(state: AgentState):
    messages = [
        SystemMessage(content=itinerary_optimizer_prompt),
        HumanMessage(content=f"""{state['task']}
                     \n\nHere is the research info:\n\n{state['research']}""")
    ]
    response = model.invoke(messages)
    return {"messages": [response]}

## Graph Construction
Building the agent workflow graph:
- Node addition for each specialized agent
- Edge connections with parallel execution:
    - Transport and accommodation advisors run in parallel after destination planning
- Conditional routing based on user needs:
    - Plan creation path (when 'Plan:' is detected)
    - Plan refinement path (when 'Refine:' is detected) 
    - Assistant path (continues chat interaction)

In [ ]:
graph = StateGraph(AgentState)

graph.add_node("user_guide", user_guide_node)
graph.add_node("itinerary_researcher", itinerary_researcher_node)
graph.add_node("itinerary_optimizer", itinerary_optimizer_node)
graph.add_node("destination_planner", destination_planner_node)
graph.add_node("transport_advisor", transport_advisor_node)
graph.add_node("accommodation_advisor", accommodation_advisor_node)
graph.add_node("itinerary_planner", itinerary_planner_node)

graph.add_conditional_edges(
            "user_guide",
            exists_action,
            {"plan": "destination_planner", "refine": "itinerary_researcher", "assistant": END}
        )

graph.add_edge("itinerary_researcher", "itinerary_optimizer")
graph.add_edge("itinerary_optimizer", END)
graph.add_edge("destination_planner", "transport_advisor")
graph.add_edge("destination_planner", "accommodation_advisor")
graph.add_edge("transport_advisor", "itinerary_planner")
graph.add_edge("accommodation_advisor", "itinerary_planner")
graph.add_edge("itinerary_planner", END)

graph.set_entry_point("user_guide")

agent = graph.compile(checkpointer=memory)

In [ ]:
display(Image(agent.get_graph().draw_mermaid_png()))

## Testing and Execution

### Chat Interface
Interactive chat interface with thread management

In [ ]:
def chat(user_input: str, thread: str) -> None:
    """Handle chat interactions with error handling"""
    if not user_input.strip():
        print("Error: Empty input")
        return
        
    try:
        config = {"configurable": {"thread_id": thread}}
        for event in agent.stream(
            {"messages": [{"role": "user", "content": user_input}]},
            config
        ):
            for value in event.values():
                if "messages" in value:
                    print(f"Assistant: {value['messages'][-1].content}")
                elif "task" in value:
                    print("Assistant: Preparing your travel itinerary...")
    except Exception as e:
        print(f"Chat Error: {e}")
        print("Please try again or start a new session.")


thread=""
while True:
  if not thread:
    thread = input("chat_id: ")

  user_input = input("User: ")

  if user_input.lower() in ["quit", "exit", "q"]:
      print("Goodbye!")
      break

  chat(user_input, thread)

### State Inspection

In [ ]:
agent.get_state(config={"configurable": {"thread_id": "Istanbul"}}).values